simple code to detect and draw bounding boxes

In [2]:
import ultralytics
import numpy as np
import pandas as pd
from PIL import Image
import pickle
import cv2



In [3]:
with open('cached_model.pkl','rb') as f:
    model = pickle.load(f)

In [4]:
image = Image.open('traffic.jpg')

In [26]:
results = model(image)


0: 352x640 45 cars, 54.1ms
Speed: 8.1ms preprocess, 54.1ms inference, 0.6ms postprocess per image at shape (1, 3, 352, 640)


In [27]:
image = np.array(image)

In [28]:
data = results[0].boxes

In [30]:
# we get bounding box coordinates from results.data.boxes.xyxy
for x,y,w,h in data.xyxy.tolist():
    x= int(x)
    y =int(y)
    w= int(w)
    h= int(h)
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),2)
cv2.imshow('image',image)
cv2.waitKey(0)
cv2.destroyAllWindows()


detect in video


In [33]:

cap = cv2.VideoCapture('highway_mini.mp4')

while True:
    ret,frame =cap.read()
    if not ret:
        print('breaking')
        break
    frame = cv2.resize(frame,(1020,500))
    results = model(frame)
    data = results[0].boxes
    for x,y,w,h in data.xyxy.tolist():
        x= int(x)
        y =int(y)
        w= int(w)
        h= int(h)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
    cv2.imshow('frame',frame)
    if cv2.waitKey(0)&0xFF==27:
      break
cap.release()
cv2.destroyAllWindows()
        
    


0: 320x640 7 cars, 1 bus, 2 trucks, 2.4ms
Speed: 8.9ms preprocess, 2.4ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 1 bus, 1 truck, 2.4ms
Speed: 1.0ms preprocess, 2.4ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 1 bus, 2 trucks, 2.2ms
Speed: 0.8ms preprocess, 2.2ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 2 trucks, 2.7ms
Speed: 0.8ms preprocess, 2.7ms inference, 0.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 1 truck, 4.2ms
Speed: 1.6ms preprocess, 4.2ms inference, 0.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 7.0ms
Speed: 2.4ms preprocess, 7.0ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 8.5ms
Speed: 2.6ms preprocess, 8.5ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 3.4ms
Speed: 1.3ms preproce

using tracker class

In [6]:
from tracker import*
tracker = Tracker()
cap = cv2.VideoCapture('highway_mini.mp4')

while True:
    ret,frame =cap.read()
    if not ret:
        print('breaking')
        break
    frame = cv2.resize(frame,(1020,500))
    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")
    list = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        list.append([x1, y1, x2, y2])
        bbox_id = tracker.update(list)
        for bbox in bbox_id:
            x3, y3, x4, y4, id = bbox
            cx = int(x3 + x4) // 2
            cy = int(y3 + y4) // 2
            cv2.rectangle(frame,(x3,y3),(x4,y4),(0,0,255),2)
    cv2.imshow('frame',frame)
    if cv2.waitKey(0)&0xFF==27:
      break
cap.release()
cv2.destroyAllWindows()


0: 320x640 7 cars, 1 bus, 2 trucks, 2.2ms
Speed: 7.6ms preprocess, 2.2ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 1 bus, 1 truck, 5.0ms
Speed: 0.8ms preprocess, 5.0ms inference, 1.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 1 bus, 2 trucks, 2.1ms
Speed: 0.8ms preprocess, 2.1ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 2 trucks, 2.1ms
Speed: 0.8ms preprocess, 2.1ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 1 truck, 4.6ms
Speed: 0.8ms preprocess, 4.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 3.6ms
Speed: 1.5ms preprocess, 3.6ms inference, 0.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 2.4ms
Speed: 1.4ms preprocess, 2.4ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 6.7ms
Speed: 0.8ms preproce